In [1]:
# !pip install transformers==4.10.2
# !pip install sentencepiece==0.1.96

In [26]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    T5ForConditionalGeneration,
    MT5ForConditionalGeneration,
    ByT5Tokenizer,
    PreTrainedTokenizer,
    T5TokenizerFast as T5Tokenizer,
    MT5TokenizerFast as MT5Tokenizer,
)
from transformers import AutoTokenizer
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelWithLMHead, AutoTokenizer
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from transformers import RobertaTokenizer

torch.cuda.empty_cache()
pl.seed_everything(42)


class PyTorchDataModule(Dataset):
    """  PyTorch Dataset class  """

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: PreTrainedTokenizer,
        source_max_token_len: int = 512,
        target_max_token_len: int = 512,
    ):
        """
        initiates a PyTorch Dataset Module for input data
        Args:
            data (pd.DataFrame): input pandas dataframe. Dataframe must have 2 column --> "source_text" and "target_text"
            tokenizer (PreTrainedTokenizer): a PreTrainedTokenizer (T5Tokenizer, MT5Tokenizer, or ByT5Tokenizer)
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
        """
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        """ returns length of data """
        return len(self.data)

    def __getitem__(self, index: int):
        """ returns dictionary of input tensors to feed into T5/MT5 model"""

        data_row = self.data.iloc[index]
        source_text = data_row["source_text"]

        source_text_encoding = self.tokenizer(
            source_text,
            max_length=self.source_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        target_text_encoding = self.tokenizer(
            data_row["target_text"],
            max_length=self.target_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        labels = target_text_encoding["input_ids"]
        labels[
            labels == 0
        ] = -100  # to make sure we have correct labels for T5 text generation

        return dict(
            source_text_input_ids=source_text_encoding["input_ids"].flatten(),
            source_text_attention_mask=source_text_encoding["attention_mask"].flatten(),
            labels=labels.flatten(),
            labels_attention_mask=target_text_encoding["attention_mask"].flatten(),
        )


class LightningDataModule(pl.LightningDataModule):
    """ PyTorch Lightning data class """

    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: PreTrainedTokenizer,
        batch_size: int = 4,
        source_max_token_len: int = 512,
        target_max_token_len: int = 512,
        num_workers: int = 2,
    ):
        """
        initiates a PyTorch Lightning Data Module
        Args:
            train_df (pd.DataFrame): training dataframe. Dataframe must contain 2 columns --> "source_text" & "target_text"
            test_df (pd.DataFrame): validation dataframe. Dataframe must contain 2 columns --> "source_text" & "target_text"
            tokenizer (PreTrainedTokenizer): PreTrainedTokenizer (T5Tokenizer, MT5Tokenizer, or ByT5Tokenizer)
            batch_size (int, optional): batch size. Defaults to 4.
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
        """
        super().__init__()

        self.train_df = train_df
        self.test_df = test_df
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len
        self.num_workers = num_workers

    def setup(self, stage=None):
        self.train_dataset = PyTorchDataModule(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len,
        )
        self.test_dataset = PyTorchDataModule(
            self.test_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len,
        )

    def train_dataloader(self):
        """ training dataloader """
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        """ test dataloader """
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

    def val_dataloader(self):
        """ validation dataloader """
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )


class LightningModel(pl.LightningModule):
    """ PyTorch Lightning Model class"""

    def __init__(
        self,
        tokenizer,
        model,
        outputdir: str = "outputs",
        save_only_last_epoch: bool = False,
    ):
        """
        initiates a PyTorch Lightning Model
        Args:
            tokenizer : T5/MT5/ByT5 tokenizer
            model : T5/MT5/ByT5 model
            outputdir (str, optional): output directory to save model checkpoints. Defaults to "outputs".
            save_only_last_epoch (bool, optional): If True, save just the last epoch else models are saved for every epoch
        """
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.outputdir = outputdir
        self.average_training_loss = None
        self.average_validation_loss = None
        self.save_only_last_epoch = save_only_last_epoch

    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        """ forward step """
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask,
        )

        return output.loss, output.logits

    def training_step(self, batch, batch_size):
        """ training step """
        input_ids = batch["source_text_input_ids"]
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log(
            "train_loss", loss, prog_bar=True, logger=True, on_epoch=True, on_step=True
        )
        return loss

    def validation_step(self, batch, batch_size):
        """ validation step """
        input_ids = batch["source_text_input_ids"]
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log(
            "val_loss", loss, prog_bar=True, logger=True, on_epoch=True, on_step=True
        )
        return loss

    def test_step(self, batch, batch_size):
        """ test step """
        input_ids = batch["source_text_input_ids"]
        attention_mask = batch["source_text_attention_mask"]
        labels = batch["labels"]
        labels_attention_mask = batch["labels_attention_mask"]

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels,
        )

        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        """ configure optimizers """
        return AdamW(self.parameters(), lr=0.0001)

    def training_epoch_end(self, training_step_outputs):
        """ save tokenizer and model on epoch end """
        self.average_training_loss = np.round(
            torch.mean(torch.stack([x["loss"] for x in training_step_outputs])).item(),
            4,
        )
        path = f"{self.outputdir}/simplet5-epoch-{self.current_epoch}-train-loss-{str(self.average_training_loss)}-val-loss-{str(self.average_validation_loss)}"
        if self.save_only_last_epoch:
            if self.current_epoch == self.trainer.max_epochs - 1:
                self.tokenizer.save_pretrained(path)
                self.model.save_pretrained(path)
        else:
            self.tokenizer.save_pretrained(path)
            self.model.save_pretrained(path)

    def validation_epoch_end(self, validation_step_outputs):
        _loss = [x.cpu() for x in validation_step_outputs]
        self.average_validation_loss = np.round(
            torch.mean(torch.stack(_loss)).item(),
            4,
        )


class SimpleT5:
    """ Custom SimpleT5 class """

    def __init__(self) -> None:
        """ initiates SimpleT5 class """
        pass

    def from_pretrained(self, model_type="t5", model_name="t5-base") -> None:
        """
        loads T5/MT5 Model model for training/finetuning
        Args:
            model_type (str, optional): "t5" or "mt5" . Defaults to "t5".
            model_name (str, optional): exact model architecture name, "t5-base" or "t5-large". Defaults to "t5-base".
        """
        if model_type == "t5":
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_name}")
            self.model = T5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(f"{model_name}")
            self.model = MT5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type == "byt5":
            self.tokenizer = ByT5Tokenizer.from_pretrained(f"{model_name}")
            self.model = T5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
        elif model_type =="CodeT5":
#             ("Salesforce/codet5-base")
            self.tokenizer = RobertaTokenizer.from_pretrained(f"{model_name}")
#             T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")
            self.model = T5ForConditionalGeneration.from_pretrained(
                f"{model_name}", return_dict=True
            )
            

    def train(
        self,
        train_df: pd.DataFrame,
        eval_df: pd.DataFrame,
        source_max_token_len: int = 512,
        target_max_token_len: int = 512,
        batch_size: int = 8,
        max_epochs: int = 5,
        use_gpu: bool = True,
        outputdir: str = "outputs",
        early_stopping_patience_epochs: int = 0,  # 0 to disable early stopping feature
        precision=32,
        logger="default",
        dataloader_num_workers: int = 2,
        save_only_last_epoch: bool = False,
    ):
        """
        trains T5/MT5 model on custom dataset
        Args:
            train_df (pd.DataFrame): training datarame. Dataframe must have 2 column --> "source_text" and "target_text"
            eval_df ([type], optional): validation datarame. Dataframe must have 2 column --> "source_text" and "target_text"
            source_max_token_len (int, optional): max token length of source text. Defaults to 512.
            target_max_token_len (int, optional): max token length of target text. Defaults to 512.
            batch_size (int, optional): batch size. Defaults to 8.
            max_epochs (int, optional): max number of epochs. Defaults to 5.
            use_gpu (bool, optional): if True, model uses gpu for training. Defaults to True.
            outputdir (str, optional): output directory to save model checkpoints. Defaults to "outputs".
            early_stopping_patience_epochs (int, optional): monitors val_loss on epoch end and stops training, if val_loss does not improve after the specied number of epochs. set 0 to disable early stopping. Defaults to 0 (disabled)
            precision (int, optional): sets precision training - Double precision (64), full precision (32) or half precision (16). Defaults to 32.
            logger (pytorch_lightning.loggers) : any logger supported by PyTorch Lightning. Defaults to "default". If "default", pytorch lightning default logger is used.
            dataloader_num_workers (int, optional): number of workers in train/test/val dataloader
            save_only_last_epoch (bool, optional): If True, saves only the last epoch else models are saved at every epoch
        """
        self.data_module = LightningDataModule(
            train_df,
            eval_df,
            self.tokenizer,
            batch_size=batch_size,
            source_max_token_len=source_max_token_len,
            target_max_token_len=target_max_token_len,
            num_workers=dataloader_num_workers,
        )

        self.T5Model = LightningModel(
            tokenizer=self.tokenizer,
            model=self.model,
            outputdir=outputdir,
            save_only_last_epoch=save_only_last_epoch,
        )

        # add callbacks
        callbacks = [TQDMProgressBar(refresh_rate=5)]

        if early_stopping_patience_epochs > 0:
            early_stop_callback = EarlyStopping(
                monitor="val_loss",
                min_delta=0.00,
                patience=early_stopping_patience_epochs,
                verbose=True,
                mode="min",
            )
            callbacks.append(early_stop_callback)

        # add gpu support
        gpus = 1 if use_gpu else 0

        # add logger
        loggers = True if logger == "default" else logger

        # prepare trainer
        trainer = pl.Trainer(
            logger=loggers,
            callbacks=callbacks,
            max_epochs=max_epochs,
            gpus=gpus,
            precision=precision,
            log_every_n_steps=1,
        )

        # fit trainer
        trainer.fit(self.T5Model, self.data_module)

    def load_model(
        self, model_type: str = "t5", model_dir: str = "outputs", use_gpu: bool = False
    ):
        """
        loads a checkpoint for inferencing/prediction
        Args:
            model_type (str, optional): "t5" or "mt5". Defaults to "t5".
            model_dir (str, optional): path to model directory. Defaults to "outputs".
            use_gpu (bool, optional): if True, model uses gpu for inferencing/prediction. Defaults to True.
        """
        if model_type == "t5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = T5Tokenizer.from_pretrained(f"{model_dir}")
        elif model_type == "mt5":
            self.model = MT5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = MT5Tokenizer.from_pretrained(f"{model_dir}")
        elif model_type == "byt5":
            self.model = T5ForConditionalGeneration.from_pretrained(f"{model_dir}")
            self.tokenizer = ByT5Tokenizer.from_pretrained(f"{model_dir}")

        if use_gpu:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                raise "exception ---> no gpu found. set use_gpu=False, to use CPU"
        else:
            self.device = torch.device("cpu")

        self.model = self.model.to(self.device)

    def predict(
        self,
        source_text: str,
        max_length: int = 512,
        num_return_sequences: int = 1,
        num_beams: int = 2,
        top_k: int = 50,
        top_p: float = 0.95,
        do_sample: bool = True,
        repetition_penalty: float = 2.5,
        length_penalty: float = 1.0,
        early_stopping: bool = True,
        skip_special_tokens: bool = True,
        clean_up_tokenization_spaces: bool = True,
    ):
        """
        generates prediction for T5/MT5 model
        Args:
            source_text (str): any text for generating predictions
            max_length (int, optional): max token length of prediction. Defaults to 512.
            num_return_sequences (int, optional): number of predictions to be returned. Defaults to 1.
            num_beams (int, optional): number of beams. Defaults to 2.
            top_k (int, optional): Defaults to 50.
            top_p (float, optional): Defaults to 0.95.
            do_sample (bool, optional): Defaults to True.
            repetition_penalty (float, optional): Defaults to 2.5.
            length_penalty (float, optional): Defaults to 1.0.
            early_stopping (bool, optional): Defaults to True.
            skip_special_tokens (bool, optional): Defaults to True.
            clean_up_tokenization_spaces (bool, optional): Defaults to True.
        Returns:
            list[str]: returns predictions
        """
        input_ids = self.tokenizer.encode(
            source_text, return_tensors="pt", add_special_tokens=True
        )
        input_ids = input_ids.to(self.device)
        generated_ids = self.model.generate(
            input_ids=input_ids,
            num_beams=num_beams,
            max_length=max_length,
            repetition_penalty=repetition_penalty,
            length_penalty=length_penalty,
            early_stopping=early_stopping,
            top_p=top_p,
            top_k=top_k,
            num_return_sequences=num_return_sequences,
        )
        preds = [
            self.tokenizer.decode(
                g,
                skip_special_tokens=skip_special_tokens,
                clean_up_tokenization_spaces=clean_up_tokenization_spaces,
            )
            for g in generated_ids
        ]
        return preds

Global seed set to 42


In [27]:
import pandas as pd
# df = pd.read_csv('/home/chirayu.tripathi/t5/data_input_questions.csv')
df = pd.read_csv('/home/chirayu.tripathi/t5/active_passive/paraphrased_new.csv')

In [28]:
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,pandas,tables,input
0,department_management,SELECT count(*) FROM head WHERE age > 56,"['SELECT', 'count', '(', '*', ')', 'FROM', 'he...","['select', 'count', '(', '*', ')', 'from', 'he...",How many heads of the departments are older th...,"['How', 'many', 'heads', 'of', 'the', 'departm...","{'from': {'table_units': [['table_unit', 1]], ...",(head['age'] > 56).sum(),"{0: {'department': ['Department_ID', 'Name', '...",how many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","['SELECT', 'name', ',', 'born_state', ',', 'ag...","['select', 'name', ',', 'born_state', ',', 'ag...","List the name, born state and age of the heads...","['List', 'the', 'name', ',', 'born', 'state', ...","{'from': {'table_units': [['table_unit', 1]], ...","head[['name', 'born_state', 'age']].sort_value...","{0: {'department': ['Department_ID', 'Name', '...","list the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","['SELECT', 'creation', ',', 'name', ',', 'budg...","['select', 'creation', ',', 'name', ',', 'budg...","List the creation year, name and budget of eac...","['List', 'the', 'creation', 'year', ',', 'name...","{'from': {'table_units': [['table_unit', 0]], ...","department[['creation', 'name', 'budget_in_bil...","{0: {'department': ['Department_ID', 'Name', '...","list the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","['SELECT', 'max', '(', 'budget_in_billions', '...","['select', 'max', '(', 'budget_in_billions', '...",What are the maximum and minimum budget of the...,"['What', 'are', 'the', 'maximum', 'and', 'mini...","{'from': {'table_units': [['table_unit', 0]], ...","department['budget_in_billions'].agg(['max', '...","{0: {'department': ['Department_ID', 'Name', '...",what are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,"['SELECT', 'avg', '(', 'num_employees', ')', '...","['select', 'avg', '(', 'num_employees', ')', '...",What is the average number of employees of the...,"['What', 'is', 'the', 'average', 'number', 'of...","{'from': {'table_units': [['table_unit', 0]], ...",department.loc[lambda x: x['ranking'].between(...,"{0: {'department': ['Department_ID', 'Name', '...",what is the average number of employees of the...
...,...,...,...,...,...,...,...,...,...,...
19618,insurance_fnol,"SELECT t2.date_opened , t2.date_closed FROM c...","['SELECT', 't2.date_opened', ',', 't2.date_clo...","['select', 't2', '.', 'date_opened', ',', 't2'...",Find all the policies associated with the cust...,"['Retrieve', 'the', 'open', 'and', 'close', 'd...","{'from': {'table_units': [['table_unit', 0], [...","customers.merge(customers_policies, on='custom...","{0: {'Customers': ['Customer_ID', 'Customer_na...",find all the policies associated with the cust...
19619,insurance_fnol,"SELECT t2.date_opened , t2.date_closed FROM c...","['SELECT', 't2.date_opened', ',', 't2.date_clo...","['select', 't2', '.', 'date_opened', ',', 't2'...",The open and close dates of all policies used ...,"['What', 'are', 'the', 'open', 'and', 'close',...","{'from': {'table_units': [['table_unit', 0], [...","customers.merge(customers_policies, on='custom...","{0: {'Customers': ['Customer_ID', 'Customer_na...",the open and close dates of all policies used ...
19620,medicine_enzyme_interaction,SELECT count(*) FROM enzyme,"['SELECT', 'count', '(', '*', ')', 'FROM', 'en...","['select', 'count', '(', '*', ')', 'from', 'en...",The total count of enzymes are.,"['What', 'is', 'the', 'total', 'count', 'of', ...","{'from': {'table_units': [['table_unit', 1]], ...",enzyme.shape[0],"{0: {'medicine': ['id', 'name', 'Trade_Name', ...","the total count of enzymes are. | enzyme : id,..."
19621,medicine_enzyme_interaction,SELECT name FROM enzyme ORDER BY name DE

In [29]:
df1 = pd.read_csv('/home/chirayu.tripathi/t5/active_passive/train.csv')

In [30]:
df

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,pandas,tables,input
0,department_management,SELECT count(*) FROM head WHERE age > 56,"['SELECT', 'count', '(', '*', ')', 'FROM', 'he...","['select', 'count', '(', '*', ')', 'from', 'he...",How many heads of the departments are older th...,"['How', 'many', 'heads', 'of', 'the', 'departm...","{'from': {'table_units': [['table_unit', 1]], ...",(head['age'] > 56).sum(),"{0: {'department': ['Department_ID', 'Name', '...",how many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","['SELECT', 'name', ',', 'born_state', ',', 'ag...","['select', 'name', ',', 'born_state', ',', 'ag...","List the name, born state and age of the heads...","['List', 'the', 'name', ',', 'born', 'state', ...","{'from': {'table_units': [['table_unit', 1]], ...","head[['name', 'born_state', 'age']].sort_value...","{0: {'department': ['Department_ID', 'Name', '...","list the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","['SELECT', 'creation', ',', 'name', ',', 'budg...","['select', 'creation', ',', 'name', ',', 'budg...","List the creation year, name and budget of eac...","['List', 'the', 'creation', 'year', ',', 'name...","{'from': {'table_units': [['table_unit', 0]], ...","department[['creation', 'name', 'budget_in_bil...","{0: {'department': ['Department_ID', 'Name', '...","list the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","['SELECT', 'max', '(', 'budget_in_billions', '...","['select', 'max', '(', 'budget_in_billions', '...",What are the maximum and minimum budget of the...,"['What', 'are', 'the', 'maximum', 'and', 'mini...","{'from': {'table_units': [['table_unit', 0]], ...","department['budget_in_billions'].agg(['max', '...","{0: {'department': ['Department_ID', 'Name', '...",what are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,"['SELECT', 'avg', '(', 'num_employees', ')', '...","['select', 'avg', '(', 'num_employees', ')', '...",What is the average number of employees of the...,"['What', 'is', 'the', 'average', 'number', 'of...","{'from': {'table_units': [['table_unit', 0]], ...",department.loc[lambda x: x['ranking'].between(...,"{0: {'department': ['Department_ID', 'Name', '...",what is the average number of employees of the...
...,...,...,...,...,...,...,...,...,...,...
19618,insurance_fnol,"SELECT t2.date_opened , t2.date_closed FROM c...","['SELECT', 't2.date_opened', ',', 't2.date_clo...","['select', 't2', '.', 'date_opened', ',', 't2'...",Find all the policies associated with the cust...,"['Retrieve', 'the', 'open', 'and', 'close', 'd...","{'from': {'table_units': [['table_unit', 0], [...","customers.merge(customers_policies, on='custom...","{0: {'Customers': ['Customer_ID', 'Customer_na...",find all the policies associated with the cust...
19619,insurance_fnol,"SELECT t2.date_opened , t2.date_closed FROM c...","['SELECT', 't2.date_opened', ',', 't2.date_clo...","['select', 't2', '.', 'date_opened', ',', 't2'...",The open and close dates of all policies used ...,"['What', 'are', 'the', 'open', 'and', 'close',...","{'from': {'table_units': [['table_unit', 0], [...","customers.merge(customers_policies, on='custom...","{0: {'Customers': ['Customer_ID', 'Customer_na...",the open and close dates of all policies used ...
19620,medicine_enzyme_interaction,SELECT count(*) FROM enzyme,"['SELECT', 'count', '(', '*', ')', 'FROM', 'en...","['select', 'count', '(', '*', ')', 'from', 'en...",The total count of enzymes are.,"['What', 'is', 'the', 'total', 'count', 'of', ...","{'from': {'table_units': [['table_unit', 1]], ...",enzyme.shape[0],"{0: {'medicine': ['id', 'name', 'Trade_Name', ...","the total count of enzymes are. | enzyme : id,..."
19621,medicine_enzyme_interaction,SELECT name FROM enzyme ORDER BY name DE

In [31]:
df['source'] = 'pandas: '+df['input'].apply(lambda x:x.lower())
# df['source'] = df['question'].apply(lambda x:x.lower())

In [32]:
df['target'] = df['pandas'].apply(lambda x:x.lower())

In [33]:
# for i,j in df.iterrows():
#     print(j['pandas'])
#     print(j['target_text'])
#     print('-'*20)

In [34]:
df = df.rename(columns={"target":"target_text", "source":"source_text"})

In [35]:
df[df['question']=='what is the venue of the competition "1994 fifa world cup qualification" hosted by "nanjing ( jiangsu )"']

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,pandas,tables,input,source_text,target_text


In [36]:
df = df[['source_text','target_text']]

In [37]:
df

,source_text,target_text
0,pandas: how many heads of the departments are ...,(head['age'] > 56).sum()
1,"pandas: list the name, born state and age of t...","head[['name', 'born_state', 'age']].sort_value..."
2,"pandas: list the creation year, name and budge...","department[['creation', 'name', 'budget_in_bil..."
3,pandas: what are the maximum and minimum budge...,"department['budget_in_billions'].agg(['max', '..."
4,pandas: what is the average number of employee...,department.loc[lambda x: x['ranking'].between(...
...,...,...
19618,pandas: find all the policies associated with ...,"customers.merge(customers_policies, on='custom..."
19619,pandas: the open and close dates of all polici...,"customers.merge(customers_policies, on='custom..."
19620,pandas: the total count of enzymes are. | enzy...,enzyme.shape[0]
19621,pandas: list the names of enzymes in descendin...,"enzyme.sort_values('name', ascending=false)['n..."


In [38]:
df1

,source_text,target_text
0,code: how big is new mexico | state : state_na...,"state[state['state_name'] == ""new mexico""]['ar..."
1,code: what is the area of new mexico | state :...,"state[state['state_name'] == ""new mexico""]['ar..."
2,code: how large is new mexico | state : state_...,"state[state['state_name'] == ""new mexico""]['ar..."
3,code: what is the area of the new mexico state...,"state[state['state_name'] == ""new mexico""]['ar..."
4,code: what is the size of new mexico | state :...,"state[state['state_name'] == ""new mexico""]['ar..."
...,...,...
4467,code: what is the name of the singer with the ...,"singer[['name', 'net_worth_millions']].sort_va..."
4468,code: what is the name of the singer who is wo...,"singer[['name', 'net_worth_millions']].sort_va..."
4469,code: please show the most common citizenship ...,singer.groupby('citizenship').agg(col2 = ('cit...
4470,code: what is the most common singer citizensh...,singer.groupby('citizenship').agg(col2 = ('cit...


In [39]:
df1['source_text'] = df1['source_text'].apply(lambda x:x.replace('code: ','pandas: '))

In [40]:
df1

,source_text,target_text
0,pandas: how big is new mexico | state : state_...,"state[state['state_name'] == ""new mexico""]['ar..."
1,pandas: what is the area of new mexico | state...,"state[state['state_name'] == ""new mexico""]['ar..."
2,pandas: how large is new mexico | state : stat...,"state[state['state_name'] == ""new mexico""]['ar..."
3,pandas: what is the area of the new mexico sta...,"state[state['state_name'] == ""new mexico""]['ar..."
4,pandas: what is the size of new mexico | state...,"state[state['state_name'] == ""new mexico""]['ar..."
...,...,...
4467,pandas: what is the name of the singer with th...,"singer[['name', 'net_worth_millions']].sort_va..."
4468,pandas: what is the name of the singer who is ...,"singer[['name', 'net_worth_millions']].sort_va..."
4469,pandas: please show the most common citizenshi...,singer.groupby('citizenship').agg(col2 = ('cit...
4470,pandas: what is the most common singer citizen...,singer.groupby('citizenship').agg(col2 = ('cit...


In [41]:
df = pd.concat([df1,df])

In [42]:
df.drop_duplicates(subset=['source_text'],inplace=True)

In [43]:
df

,source_text,target_text
0,pandas: how big is new mexico | state : state_...,"state[state['state_name'] == ""new mexico""]['ar..."
1,pandas: what is the area of new mexico | state...,"state[state['state_name'] == ""new mexico""]['ar..."
2,pandas: how large is new mexico | state : stat...,"state[state['state_name'] == ""new mexico""]['ar..."
3,pandas: what is the area of the new mexico sta...,"state[state['state_name'] == ""new mexico""]['ar..."
4,pandas: what is the size of new mexico | state...,"state[state['state_name'] == ""new mexico""]['ar..."
...,...,...
19618,pandas: find all the policies associated with ...,"customers.merge(customers_policies, on='custom..."
19619,pandas: the open and close dates of all polici...,"customers.merge(customers_policies, on='custom..."
19620,pandas: the total count of enzymes are. | enzy...,enzyme.shape[0]
19621,pandas: list the names of enzymes in descendin...,"enzyme.sort_values('name', ascending=false)['n..."


In [44]:
df['source_text'] = df['source_text'].apply(lambda x: x.replace(' | ',' </s> '))

In [45]:
df

,source_text,target_text
0,pandas: how big is new mexico </s> state : sta...,"state[state['state_name'] == ""new mexico""]['ar..."
1,pandas: what is the area of new mexico </s> st...,"state[state['state_name'] == ""new mexico""]['ar..."
2,pandas: how large is new mexico </s> state : s...,"state[state['state_name'] == ""new mexico""]['ar..."
3,pandas: what is the area of the new mexico sta...,"state[state['state_name'] == ""new mexico""]['ar..."
4,pandas: what is the size of new mexico </s> st...,"state[state['state_name'] == ""new mexico""]['ar..."
...,...,...
19618,pandas: find all the policies associated with ...,"customers.merge(customers_policies, on='custom..."
19619,pandas: the open and close dates of all polici...,"customers.merge(customers_policies, on='custom..."
19620,pandas: the total count of enzymes are. </s> e...,enzyme.shape[0]
19621,pandas: list the names of enzymes in descendin...,"enzyme.sort_values('name', ascending=false)['n..."


In [46]:
df = df.sample(frac=1).reset_index(drop=True)

In [47]:
df

,source_text,target_text
0,pandas: what are the names of the authors in a...,book.sort_values('writer')['writer']
1,pandas: give me the budget of the movie juno f...,movie.loc[(movie['release_year']==2007) & (mov...
2,pandas: what are the ids of all moviest hat ha...,"rating[~rating['rid'].isin(pd.merge(rating, re..."
3,pandas: what are the names of the customers wh...,"pd.merge(orders, customers, on='customer_id')...."
4,pandas: what are the names of those students w...,"pd.merge(student, takes, on='id').loc[lambda x..."
...,...,...
19573,"pandas: for each user, return its name and the...","useracct.merge(review, on='u_id').groupby('nam..."
19574,pandas: how many males and females are overwei...,people[people['weight'] > 85].groupby('sex').s...
19575,pandas: what are the names of parties and thei...,"pd.merge(party, region, on='region_id')[['part..."
19576,pandas: what are the durations of the longest ...,"track.agg({'milliseconds': ['max', 'min']})"


In [25]:
# df.to_csv('/home/chirayu.tripathi/t5/previously_merged.csv',index=False)

In [23]:
# df_train = df_train.rename(columns={"sentiment":"target_text", "review":"source_text"})
# #df = df[['source_text', 'target_text']]
# df_eval = df_eval.rename(columns={"sentiment":"target_text", "review":"source_text"})

In [48]:
df_train = df.iloc[:15000][['source_text','target_text']]
df_eval = df.iloc[15000:][['source_text','target_text']]
# df_train = df.iloc[:8000][['source_text','target_text']]
# df_eval = df.iloc[8000:][['source_text','target_text']]


In [49]:
df_train['source_text'][0]

'pandas: what are the names of the authors in ascending alphabetical order? </s> book : book_id, title, issues, writer'

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model = SimpleT5()
model.from_pretrained(model_type="CodeT5", model_name="Salesforce/codet5-large")
model.train(train_df=df_train, 
            eval_df=df_eval,
            source_max_token_len=128,
            target_max_token_len=128, 
            batch_size=8, max_epochs=6, use_gpu=True)

Downloading:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/288k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Set SLURM handle signals.

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 737 M 
-----------------------------------------------------
737 M     Trainable params
0         Non-trainable params
737 M     Total params
2,950.558 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/chirayu.tripathi/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
/home/chirayu.tripathi/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# model = AutoModelForSeq2SeqLM.from_pretrained("/home/chirayu.tripathi/t5/baseline/simplet5-epoch-3-train-loss-0.1266-val-loss-0.1738")
# tokenizer = AutoTokenizer.from_pretrained("/home/chirayu.tripathi/t5/baseline/simplet5-epoch-3-train-loss-0.1266-val-loss-0.1738")
# 8
model = AutoModelForSeq2SeqLM.from_pretrained("/home/chirayu.tripathi/t5/baseline/simplet5-epoch-8-train-loss-0.0457-val-loss-0.1048")
tokenizer = AutoTokenizer.from_pretrained("/home/chirayu.tripathi/t5/baseline/simplet5-epoch-8-train-loss-0.0457-val-loss-0.1048")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def get_query(text, num_beams=2,max_length=128, repetition_penalty=2.5, length_penalty=1, early_stopping=True,top_p=.95, top_k=50, num_return_sequences=1):
  
  input_ids = tokenizer.encode(
    text, return_tensors="pt", add_special_tokens=True
  )
  
  input_ids = input_ids.to(device)
  generated_ids = model.generate(
      input_ids=input_ids,
     
      num_beams=num_beams,
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      length_penalty=length_penalty,
      early_stopping=early_stopping,
      top_p=top_p,
      top_k=top_k,
      num_return_sequences=num_return_sequences,
  )
  query = [tokenizer.decode(generated_id,skip_special_tokens=True,clean_up_tokenization_spaces=True,) for generated_id in generated_ids]
  return query[0]

In [21]:
import pandas as pd
titanic = pd.read_csv('/home/chirayu.tripathi/t5/titanic.csv')

In [22]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [25]:
st = '''pandas: list all the female passengers
</s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked'''
get_query(st)

'titanic[titanic[\'sex\'] == "f"][\'passengerid\']'

In [70]:
st = '''pandas: find the names of peoples with age 32 and containing 'bill' in their name? 
</s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked'''
get_query(st)

"titanic[(titanic['age'] == 32) & (titanic['name'].str.contains('bill')]['name']"

In [38]:
st = '''pandas: find the names of peoples with age of 32 and contains 'bill' in their name? </s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked'''
get_query(st)

"titanic[(titanic['age'] == 32) & (titanic['name'].str.contains('bill'))]['name']"

In [369]:
titanic[(titanic['Age'] == 32) & (titanic['Name'].str.contains('bill'))]#['Name']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [77]:
st = '''pandas: list all tips that are greater than the minimum tip received </s> tips : total_bill, tip, sex, smoker, day, time, size'''
get_query(st)

"tips.loc[lambda x: x['tip'] > tips['tip'].min(), 'tip']"

In [28]:
tips.loc[lambda x: x['tip'] > tips['tip'].min(), 'tip']

0      1.01
1      1.66
2      3.50
3      3.31
4      3.61
       ... 
239    5.92
240    2.00
241    2.00
242    1.75
243    3.00
Name: tip, Length: 240, dtype: float64

In [39]:
st = '''pandas: list all the tips that are greater than the minimum tip ever received </s> tips : total_bill, tip, sex, smoker, day, time, size'''
get_query(st)

" tips.loc[lambda x: x['tip'] > tips['tip'].min(), 'tip']"

In [303]:
st = '''pandas: what is the total tip that was paid for size 2 </s> tips : total_bill, tip, sex, smoker, day, time, size'''
get_query(st)

"pd.series(tips[tips['size'] == 2]['tip'].sum())"

In [41]:
st = '''pandas: what is the sum of tip that was paid for size 2 </s> tips : total_bill, tip, sex, smoker, day, time, size'''
get_query(st)

"tips.loc[lambda x: x['size'] == 2, 'tip'].sum()"

In [93]:
st = '''pandas: what is the age of person with name 'Braund, Mr. Owen Harris' ? </s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked'''
get_query(st)

"titanic[titanic['name'] == 'braund mr. Owen harris']['age']"

In [42]:
st = '''pandas: what is the age of person with name "Braund, Mr. Owen Harris" ? </s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked'''
get_query(st)

'titanic[titanic[\'name\'] == "braund, mr. Owen harris"][\'age\']'

In [259]:
st = '''pandas: Which cabin has average age less than 21? </s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, Cabin, embarked'''
get_query(st)

"titanic.groupby('cabin').filter(lambda group: group['age'].mean() < 21).groupby('cabin')['age'].mean()"

In [43]:
st = '''pandas: which cabin has average age less than 21? </s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, Cabin, embarked'''
get_query(st)

"titanic.groupby('cabin').filter(lambda group: group['age'].mean() < 21).groupby('cabin')['age'].mean()"

In [62]:
titanic.groupby('Cabin').filter(lambda group: group['Age'].mean() < 21)['Cabin'].unique()

array(['G6', 'D47', 'F2', 'F4', 'C22 C26', 'C65', 'B57 B59 B63 B66',
       'A34', 'B79', 'C70', 'E68', 'C62 C64', 'D30', 'E121', 'D28', 'B42'],
      dtype=object)

In [34]:
titanic.groupby('Cabin').filter(lambda group: group['Age'].mean() < 21).groupby('Cabin')['Age'].mean()

Cabin
A34                 4.000000
B42                19.000000
B57 B59 B63 B66    19.500000
B79                16.000000
C22 C26             9.306667
C62 C64            18.000000
C65                17.500000
C70                17.000000
D28                16.000000
D30                19.000000
D47                19.000000
E121               16.500000
E68                18.000000
F2                 13.833333
F4                  2.500000
G6                 14.750000
Name: Age, dtype: float64

In [265]:
titanic.groupby('Cabin').filter(lambda group: group['Age'].mean() < 21).groupby('Cabin')['Age'].mean()

Cabin
A34                 4.000000
B42                19.000000
B57 B59 B63 B66    19.500000
B79                16.000000
C22 C26             9.306667
C62 C64            18.000000
C65                17.500000
C70                17.000000
D28                16.000000
D30                19.000000
D47                19.000000
E121               16.500000
E68                18.000000
F2                 13.833333
F4                  2.500000
G6                 14.750000
Name: Age, dtype: float64

In [14]:
st = '''pandas: which embarked have overall sum of survived count greater than 10? 
</s> titanic : passengerid, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked'''
get_query(st)

"titanic.loc[lambda x: x['surived'] > 10, 'employed'].sum()"

In [224]:
tips.groupby('tip').size()

tip
1.00     4
1.01     1
1.10     1
1.17     1
1.25     3
        ..
6.70     1
6.73     1
7.58     1
9.00     1
10.00    1
Length: 123, dtype: int64

In [18]:
import pandas as pd
population = pd.DataFrame({
    'country': ['America', 'Indonesia', 'France'],
    'city1': ['Chicago', 'Jakarta', 'Paris'],
    'population': [731800, 575030, 183305]
})

# Create Income DataFrame
inc = pd.DataFrame({
    'country': [ 'America', 'Indonesia', 'India', 'France', 'Greece'],
    'city2': [ 'Chicago', 'Jakarta', 'Mumbai', 'Paris', 'Yunani'],
    'income': [ 1500, 1400, 1100, 900, 1200]
    
})

In [35]:
st = '''pandas: what is the population and name of the city with highest income? 
</s> population : country, city1, population </s> inc : country, city2, income'''
get_query(st)

"pd.merge(population, inc, on='country').sort_values('income', ascending=false).iloc[0][['population', 'city1']]"

In [300]:
pd.merge(population, inc, on='country').sort_values('income', ascending=False).iloc[0][['population', 'city1']]

population     731800
city1         Chicago
Name: 0, dtype: object

In [45]:
st = '''pandas: what is the population and name of the city with highest income </s> population : country, city1, population </s> inc : country, city2, income.'''
get_query(st)

"population.merge(inc, left_on='country', right_on='country').sort_values('income', ascending=false).iloc[0][['population', 'city1']]"

In [65]:
st = '''pandas: what are the cities with population greater than the lowest population? </s> population : country, city1, population </s> inc : country, city2, income.'''
get_query(st)

"set(pd.merge(population, inc, on='country').loc[lambda x: x['population'] > population['population'].min(), 'city1'])"

In [15]:
st = '''pandas: what are the cities with population greater than the lowest population? </s> population : country, city1, population </s> inc : country, city2, income'''
get_query(st)

"set(pd.merge(population, inc, left_on='country', right_on='country').loc[lambda x: x['population'] > population['population'].min(), 'city1'])"

In [20]:
st = '''pandas: what are the cities with income greater than the average income? 
</s> population : country, city1, population 
</s> inc : country, city2, income'''
get_query(st)

"city.loc[lambda x: x['population'] > x['population'].mean(), 'city1'].unique()"

In [67]:
set(pd.merge(population, inc, on='country').loc[lambda x: x['population'] > population['population'].min(), 'city1'])

{'Chicago', 'Jakarta'}

In [272]:
pd.merge(population, income, left_on='country', right_on='country').sort_values('income', ascending=False).iloc[0]['population']

731800

In [273]:
pd.merge(population, inc, left_on='country', right_on='country').loc[lambda x: x['population'] > x.sort_values('income').iloc[0]['population']]

,country,city1,population,city2,income
0,America,New York,731800,New York,1000
1,America,New York,731800,Chicago,1500
2,Indonesia,Jakarta,575030,Jakarta,1400


In [27]:
tips = pd.read_csv('/home/chirayu.tripathi/t5/tips.csv')

In [ ]:
tips[tips['size'] == 2]['tip'].sum()

In [ ]:
tips

In [ ]:
tips.loc[lambda x: x['tip'] > tips['tip'].min(), 'tip']

In [ ]:
len(df)

In [ ]:
df_eval

In [ ]:
import os
paths = os.listdir('/home/chirayu.tripathi/t5/outputs')

In [ ]:
paths

In [ ]:
accuracy = {}

for i in paths:
    model = AutoModelForSeq2SeqLM.from_pretrained(f"/home/chirayu.tripathi/t5/outputs/{i}")
    tokenizer = AutoTokenizer.from_pretrained(f"/home/chirayu.tripathi/t5/outputs/{i}")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    count = 0
    for index,row in df_eval.iterrows():
        query = get_query(row['source_text'])
        if query.lower().replace(' ','') == row['target_text'].lower().replace(' ',''):
            count+=1
    accuracy[i] = count/len(df_eval)
#     else:
#         print(query)
#         print(row['target_text'])
#         print('-------')

In [ ]:
print('hello')

In [ ]:
count/len(df_eval)

In [ ]:
accuracy

In [ ]:

import json

json = json.dumps(accuracy)

f = open("/home/chirayu.tripathi/t5/model_optimized.json","w")

f.write(json)

f.close()


In [42]:
count

1202

In [46]:
df_eval

,source_text,target_text
7269,Pandas: how many instructors teach a course in...,teaches.loc[(teaches['semester']=='spring') & ...
7270,Pandas: what are the addresses of the course a...,course_authors_and_tutors.loc[lambda x: x['per...
7271,Pandas: what are the average score and average...,"shop[['num_of_staff', 'score']].mean()"
7272,"Pandas: in what city was "" kevin spacey "" born...",director.loc[lambda x: x['name']=='director_na...
7273,Pandas: find the titles of all the papers writ...,authors.loc[(authors['fname']=='aaron') & (aut...
...,...,...
9688,Pandas: find the id and last name of the teach...,pd.merge(detention.loc[detention['detention_ty...
9689,Pandas: what is the name of customers who do n...,customers.loc[lambda x: x['payment_method'] !=...
9690,Pandas: what is the total number of degrees gr...,degrees.loc[lambda x: x['year']>=2000].merge(c...
9691,Pandas: what are the birth date and birth plac...,"pd.merge(people, body_builder, on='people_id')..."
